In [1]:
pip install lightgbm optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 20.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# Optimal Rounder

In [3]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimzation

In [5]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'regression_l1',
                 verbose = -100,
                 boosting_type = 'gbdt', 
                 random_state = 929,
                 n_estimators = trial.suggest_int('n_estimators', 300, 10000),
                 learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log=True),
                 max_depth = trial.suggest_int('max_depth', 3, 12),
                 lambda_l1 = trial.suggest_float('lambda_l1', 0.01, 10.0, log=True),
                 lambda_l2 = trial.suggest_float('lambda_l2', 0.01, 10.0, log=True),
                 num_leaves = trial.suggest_int('num_leaves', 2, 100),
                 bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9),
                 feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9)
                 )

    
    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 929)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = LGBMRegressor(**param)
        model.fit(X_train, y_train)

        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_valid, y_valid)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [6]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-07 02:29:38,277] A new study created in memory with name: no-name-338afae0-cbfa-488d-b979-11012939aa7b


0 [LightGBM] [Warning] lambda_l1 is set=4.582742665666303, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.582742665666303
[LightGBM] [Warning] bagging_fraction is set=0.6598751261273172, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6598751261273172
[LightGBM] [Warning] feature_fraction is set=0.5939549378494506, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5939549378494506
[LightGBM] [Warning] lambda_l2 is set=0.03460127173380391, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.03460127173380391
1 [LightGBM] [Warning] lambda_l1 is set=4.582742665666303, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.582742665666303
[LightGBM] [Warning] bagging_fraction is set=0.6598751261273172, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6598751261273172
[LightGBM] [Warning] feature_fraction is set=0.5939549378494506, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5939549378494

[I 2023-02-07 02:29:57,610] Trial 0 finished with value: 0.5368140012168374 and parameters: {'n_estimators': 9296, 'learning_rate': 0.026558910851302436, 'max_depth': 6, 'lambda_l1': 4.582742665666303, 'lambda_l2': 0.03460127173380391, 'num_leaves': 37, 'bagging_fraction': 0.6598751261273172, 'feature_fraction': 0.5939549378494506}. Best is trial 0 with value: 0.5368140012168374.


0 [LightGBM] [Warning] lambda_l1 is set=0.21561964317607024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21561964317607024
[LightGBM] [Warning] bagging_fraction is set=0.3260913281683937, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3260913281683937
[LightGBM] [Warning] feature_fraction is set=0.8456116454653508, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8456116454653508
[LightGBM] [Warning] lambda_l2 is set=0.013699935120781047, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.013699935120781047
1 [LightGBM] [Warning] lambda_l1 is set=0.21561964317607024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21561964317607024
[LightGBM] [Warning] bagging_fraction is set=0.3260913281683937, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3260913281683937
[LightGBM] [Warning] feature_fraction is set=0.8456116454653508, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.845

[I 2023-02-07 02:30:08,502] Trial 1 finished with value: 0.5288062218829817 and parameters: {'n_estimators': 3208, 'learning_rate': 0.1262903061626408, 'max_depth': 7, 'lambda_l1': 0.21561964317607024, 'lambda_l2': 0.013699935120781047, 'num_leaves': 72, 'bagging_fraction': 0.3260913281683937, 'feature_fraction': 0.8456116454653508}. Best is trial 0 with value: 0.5368140012168374.


...

[I 2023-02-07 02:36:59,615] Trial 47 finished with value: 0.5725006231835612 and parameters: {'n_estimators': 415, 'learning_rate': 0.004407187882977995, 'max_depth': 9, 'lambda_l1': 0.5550321046562284, 'lambda_l2': 7.286319773639334, 'num_leaves': 57, 'bagging_fraction': 0.5268789475153901, 'feature_fraction': 0.33753876795855986}. Best is trial 46 with value: 0.5753883618519515.


0 [LightGBM] [Warning] lambda_l1 is set=0.470888859433325, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.470888859433325
[LightGBM] [Warning] bagging_fraction is set=0.5078817257920297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5078817257920297
[LightGBM] [Warning] feature_fraction is set=0.3317943644828354, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3317943644828354
[LightGBM] [Warning] lambda_l2 is set=7.221179908301856, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.221179908301856
1 [LightGBM] [Warning] lambda_l1 is set=0.470888859433325, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.470888859433325
[LightGBM] [Warning] bagging_fraction is set=0.5078817257920297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5078817257920297
[LightGBM] [Warning] feature_fraction is set=0.3317943644828354, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3317943644828354


[I 2023-02-07 02:37:02,139] Trial 48 finished with value: 0.570336369291824 and parameters: {'n_estimators': 436, 'learning_rate': 0.005238856446230232, 'max_depth': 9, 'lambda_l1': 0.470888859433325, 'lambda_l2': 7.221179908301856, 'num_leaves': 57, 'bagging_fraction': 0.5078817257920297, 'feature_fraction': 0.3317943644828354}. Best is trial 46 with value: 0.5753883618519515.


0 [LightGBM] [Warning] lambda_l1 is set=0.6242021442079363, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6242021442079363
[LightGBM] [Warning] bagging_fraction is set=0.44319481913981007, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44319481913981007
[LightGBM] [Warning] feature_fraction is set=0.40271796113998604, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40271796113998604
[LightGBM] [Warning] lambda_l2 is set=9.90978821994292, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.90978821994292
1 [LightGBM] [Warning] lambda_l1 is set=0.6242021442079363, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6242021442079363
[LightGBM] [Warning] bagging_fraction is set=0.44319481913981007, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44319481913981007
[LightGBM] [Warning] feature_fraction is set=0.40271796113998604, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40271796

[I 2023-02-07 02:37:04,801] Trial 49 finished with value: 0.5720761594665239 and parameters: {'n_estimators': 367, 'learning_rate': 0.004252974713284697, 'max_depth': 8, 'lambda_l1': 0.6242021442079363, 'lambda_l2': 9.90978821994292, 'num_leaves': 61, 'bagging_fraction': 0.44319481913981007, 'feature_fraction': 0.40271796113998604}. Best is trial 46 with value: 0.5753883618519515.


# LightGBM Modeling

In [5]:
lgb_cv_scores, lgb_imp = list(), list()
preds = list()

skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    ## Building the model
    lgb_md = LGBMRegressor(n_estimators = 1147,
                            max_depth = 10,
                            learning_rate = 0.018,
                            num_leaves = 51,
                            lambda_l1 = 0.2547,
                            lambda_l2 = 0.09305,
                            bagging_fraction = 0.6074,
                            feature_fraction = 0.2868,
                            random_state = 42).fit(X_train, Y_train)
    lgb_imp.append(lgb_md.feature_importances_)
    
    ## Predicting on X_test and test
    lgb_pred_1 = lgb_md.predict(X_test)
    lgb_pred_2 = lgb_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
    optR.fit(lgb_pred_1, Y_test)
    coef = optR.coefficients()
    lgb_pred_1 = optR.predict(lgb_pred_1, coef).astype(int)
    lgb_pred_2 = optR.predict(lgb_pred_2, coef).astype(int)
        
    ## Computing roc-auc score
    lgb_cv_scores.append(cohen_kappa_score(Y_test, lgb_pred_1, weights = 'quadratic'))
    preds.append(lgb_pred_2)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', lgb_cv_score)

[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2547
[LightGBM] [Warning] bagging_fraction is set=0.6074, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6074
[LightGBM] [Warning] feature_fraction is set=0.2868, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2868
[LightGBM] [Warning] lambda_l2 is set=0.09305, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.09305
[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2547
[LightGBM] [Warning] bagging_fraction is set=0.6074, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6074
[LightGBM] [Warning] feature_fraction is set=0.2868, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2868
[LightGBM] [Warning] lambda_l2 is set=0.09305, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.09305
[LightGBM] [Warning] lambda_l1 is set=0.2547, reg_al

In [6]:
lgb_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = lgb_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,6
3,2059,7
4,2060,6


In [7]:
submission['quality'].value_counts()

5    552
6    484
7    334
4      2
Name: quality, dtype: int64

In [8]:
submission.to_csv('LightGBM_Reg_FE_2.csv', index = False)